# Forced alignment with `Kaldi`

일반적으로 자동음성인식에서 `phoneme` 단위까지 자세하게 인식하는 경우는 연구 목적, 혹은 더 나은 `ASR model` 구축을 위한 학습 데이터 수집 목적일 경우가 많습니다. 그렇기 때문에 대부분의 경우 `WER`, 혹은 `SER` 정도를 측정하는 선에서 마무리 짓게 됩니다. 하지만 발화 실험이나 데이터 수집의 측면에서는 `ASR` 모델을 이용하여 음성 데이터를 자동 정렬하는 것이 큰 도움이 될 것이기 때문에, 아래 과정을 통해서 실제 `kaldi`를 이용하여 음성 파일을 `자동 정렬`하는 방법을 알아보도록 하겠습니다. 

**Note:** 이 `notebook`의 목적은 실제 얼마나 잘 정렬이 되었냐가 아니라 **정렬을 하는 과정을 살펴보는 것**이기 때문에, 올바른 결과를 보기 위해 `ASR` 모델을 테스트할 때 사용했던 파일들을 이용하도록 하겠습니다 (`test_dir`). 지금은 모든 디렉토리가 `하드 코딩` 되어있는 상태이므로, 나중에 다른 데이터로 모델을 실행하실 경우, 스크립트를 훑어보시고 필요한 부분은 변경하셔야 합니다.  

**Note:** 실제 `Automatic Speech Recognition`에서 제대로 단어를 측정해내는 것은 쉽지 않습니다. 이전 `notebook`에서 보신 것 처럼 단어를 완벽하게 인식하는 것도 어렵지만, [이 곳](https://www.eleanorchodroff.com/tutorial/kaldi/kaldi-forcedalignment.html)에서 한 것처럼 주어진 발음을 이용해서 단어를 예측해내는 과정은 결코 쉽지 않습니다. 이 `notebook`에서 진행하는 과정은 **낭독체 발화** 실험에서 얻을 수 있는 음성 데이터의 경우처럼 실제로 어떠한 문장을 발화하였는지 알고 있음을 전제로 합니다. 

**Note:** 일부 부분은 [이 곳](https://www.eleanorchodroff.com/tutorial/kaldi/kaldi-forcedalignment.html)을 참고하였습니다. 

In [3]:
%%bash 
head -n30 steps/align_si.sh

#!/usr/bin/env bash
# Copyright 2012  Johns Hopkins University (Author: Daniel Povey)
# Apache 2.0

# Computes training alignments using a model with delta or
# LDA+MLLT features.

# If you supply the "--use-graphs true" option, it will use the training
# graphs from the source directory (where the model is).  In this
# case the number of jobs must match with the source directory.


# Begin configuration section.  
nj=4
cmd=run.pl
use_graphs=false
# Begin configuration.
scale_opts="--transition-scale=1.0 --acoustic-scale=0.1 --self-loop-scale=0.1"
beam=10
retry_beam=40
careful=false
boost_silence=1.0 # Factor by which to boost silence during alignment.
# End configuration options.

echo "$0 $@"  # Print the command line for logging

[ -f ${KALDI_INSTRUCTIONAL_PATH}/path.sh ] && . ${KALDI_INSTRUCTIONAL_PATH}/path.sh # source the path.
. ${KALDI_INSTRUCTIONAL_PATH}/utils/parse_options.sh || exit 1;

if [ $# != 4 ]; then


In [4]:
%%bash 
mkdir exp/test_aligned

steps/align_si.sh --nj 4 data/test_dir data/lang exp/triphones_lda exp/test_aligned

steps/align_si.sh --nj 4 data/test_dir data/lang exp/triphones_lda exp/test_aligned
steps/align_si.sh: feature type is lda
steps/align_si.sh: aligning data in data/test_dir using model from exp/triphones_lda, putting alignments in exp/test_aligned
steps/align_si.sh: done aligning data.


mkdir: cannot create directory 'exp/test_aligned': File exists


## ali-to-phones

`ali-to-phone` 스크립트는 최종적으로 음성인식을 한 이후에 생성된 파일인 `ali.*.gz` 파일을 `acoustic model`을 이용하여 `CTM` 형식의 파일로 추출할 수 있게 도와줍니다. `CTM`은 **Time-Marked Conversation** file의 약어로, 파일 이름, 문장 순서, 시작 시간, 종료 시간, 그리고 발화된 음소에 대한 정보를 파일의 각 줄에 담고 있습니다. 현재 우리는 한 파일에 하나의 문장만 들어있기 때문에, 문장의 순서는 모두 *1*로 표시됩니다. 

In [8]:
%%bash
. ./path.sh
for i in exp/test_aligned/ali.*.gz; do
    ali-to-phones --ctm-output exp/test_aligned/final.mdl ark:"gunzip -c $i|" -> ${i%.gz}.ctm;
done;


ali-to-phones --ctm-output exp/test_aligned/final.mdl 'ark:gunzip -c exp/test_aligned/ali.1.gz|' - 
LOG (ali-to-phones[5.2.380~1-8e7d2]:main():ali-to-phones.cc:134) Done 63 utterances.
ali-to-phones --ctm-output exp/test_aligned/final.mdl 'ark:gunzip -c exp/test_aligned/ali.2.gz|' - 
LOG (ali-to-phones[5.2.380~1-8e7d2]:main():ali-to-phones.cc:134) Done 63 utterances.
ali-to-phones --ctm-output exp/test_aligned/final.mdl 'ark:gunzip -c exp/test_aligned/ali.3.gz|' - 
LOG (ali-to-phones[5.2.380~1-8e7d2]:main():ali-to-phones.cc:134) Done 62 utterances.
ali-to-phones --ctm-output exp/test_aligned/final.mdl 'ark:gunzip -c exp/test_aligned/ali.4.gz|' - 
LOG (ali-to-phones[5.2.380~1-8e7d2]:main():ali-to-phones.cc:134) Done 62 utterances.


In [10]:
%%bash
head exp/test_aligned/ali.1.ctm

1089-134686-0000 1 0.000 0.510 1
1089-134686-0000 1 0.510 0.060 138
1089-134686-0000 1 0.570 0.070 159
1089-134686-0000 1 0.640 0.070 138
1089-134686-0000 1 0.710 0.140 196
1089-134686-0000 1 0.850 0.070 216
1089-134686-0000 1 0.920 0.080 231
1089-134686-0000 1 1.000 0.050 90
1089-134686-0000 1 1.050 0.030 100
1089-134686-0000 1 1.080 0.060 219


보시는 것 처럼 `.ctm` 파일은 "파일 이름/문장 번호/시작 시간/종료 시간/음소"의 정보를 담고 있습니다. 음소의 경우, 이전에도 그랬듯 실제 음소가 아니라 음소의 `인덱스`로 표현되어 있습니다. 나중에 실제 음소로 변환하도록 하겠습니다. 

`kaldi`에서 `ASR`을 실행할 때 여러개의 cpu를 사용하여 작업을 진행하기 때문에, `CTM` 파일도 여러개가 있습니다. 다음의 명령어를 사용하여 하나의 파일(merged_alignment.txt)로 묶어보겠습니다. 

In [12]:
%%bash
cd exp/test_aligned
cat *.ctm > merged_alignment.txt
head merged_alignment.txt
tail merged_alignment.txt

1089-134686-0000 1 0.000 0.510 1
1089-134686-0000 1 0.510 0.060 138
1089-134686-0000 1 0.570 0.070 159
1089-134686-0000 1 0.640 0.070 138
1089-134686-0000 1 0.710 0.140 196
1089-134686-0000 1 0.850 0.070 216
1089-134686-0000 1 0.920 0.080 231
1089-134686-0000 1 1.000 0.050 90
1089-134686-0000 1 1.050 0.030 100
1089-134686-0000 1 1.080 0.060 219
1284-1181-0004 1 6.000 0.040 184
1284-1181-0004 1 6.040 0.050 87
1284-1181-0004 1 6.090 0.080 214
1284-1181-0004 1 6.170 0.050 176
1284-1181-0004 1 6.220 0.130 72
1284-1181-0004 1 6.350 0.070 32
1284-1181-0004 1 6.420 0.060 80
1284-1181-0004 1 6.480 0.030 32
1284-1181-0004 1 6.510 0.160 175
1284-1181-0004 1 6.670 0.460 1


# index2phone

`인덱스`로 표현된 음소들을 실제 음소로 바꿔보도록 하겠습니다. `data/lang/phones.txt` 파일에는 음소와 그 음소에 해당하는 인덱스 값이 나타나 있습니다. 

In [13]:
%%bash
head data/lang/phones.txt

<eps> 0
SIL 1
SIL_B 2
SIL_E 3
SIL_I 4
SIL_S 5
AA0_B 6
AA0_E 7
AA0_I 8
AA0_S 9


아래의 코드를 이용하여서 `음소-인덱스` 관계를 정의하는 `<dict>`를 만들어 보겠습니다. 

In [14]:
idx2phn = {}
with open("/scratch/kaldi/egs/INSTRUCTIONAL/data/lang/phones.txt", "r") as f:
    data = f.readlines()
    for line in data:
        line = line.strip()
        phone, idx = line.split()
        idx2phn[idx] = phone
    f.close
print(idx2phn["1"])

SIL


만들어진 `<dict>`를 이용하여 인덱스를 실제 음소로 변환한 이후, 전체 결과를 새로운 파일(**`final_ali.txt`**)에 저장하도록 하겠습니다. 

In [17]:
fout = open("/scratch/kaldi/egs/INSTRUCTIONAL/exp/test_aligned/final_ali.txt", "w")
with open("/scratch/kaldi/egs/INSTRUCTIONAL/exp/test_aligned/merged_alignment.txt", "r") as fin:
    data = fin.readlines()
    for line in data:
        line = line.strip()
        file_name, utt, start, end, idx = line.split()
        end = float(start) + float(end)
        line = file_name + " " + utt + " " + start + " " + str(end) + " " + idx
        phone = idx2phn[idx]
        result = line + " " + phone + "\n"
        fout.write(result)
    fin.close()
fout.close()

최종적으로 생성된 파일은 "파일 이름/문장 번호/시작 시간/종료 시간/음소 인덱스/실제 음소"의 정보를 담고 있습니다. 

In [18]:
%%bash
head exp/test_aligned/final_ali.txt

1089-134686-0000 1 0.000 0.51 1 SIL
1089-134686-0000 1 0.510 0.57 138 HH_B
1089-134686-0000 1 0.570 0.64 159 IY1_E
1089-134686-0000 1 0.640 0.71 138 HH_B
1089-134686-0000 1 0.710 0.85 196 OW1_I
1089-134686-0000 1 0.850 0.92 216 P_I
1089-134686-0000 1 0.920 1.0 231 T_E
1089-134686-0000 1 1.000 1.05 90 DH_B
1089-134686-0000 1 1.050 1.08 100 EH1_I
1089-134686-0000 1 1.080 1.14 219 R_E


# Create directories

실제 `TextGrid` 파일을 생성하기까지는 많은 과정을 거치게 됩니다. 나중에는 중간에 생성되는 파일/폴더들을 지우셔도 괜찮지만, 지금은 모두 남겨놓은 이후, 각각의 과정에서 어떠한 파일들이 생성되는지 살펴보겠습니다. 필요한 폴더들을 생성하겠습니다.  

In [20]:
%%bash
mkdir ${KALDI_INSTRUCTIONAL_PATH}/exp/test_aligned/align_phones
mkdir ${KALDI_INSTRUCTIONAL_PATH}/exp/test_aligned/align_prons
mkdir ${KALDI_INSTRUCTIONAL_PATH}/exp/test_aligned/align_words
mkdir ${KALDI_INSTRUCTIONAL_PATH}/exp/test_aligned/align_transcripts
mkdir ${KALDI_INSTRUCTIONAL_PATH}/exp/test_aligned/textgrid

## Separate final_ali.txt

`final_ali.txt` 하나의 파일 안에 모든 데이터에 대한 정보가 들어있기 때문에, 각각의 음성 파일에 대한 `forced alignment`를 바로 하기에는 복잡한 과정이 필요합니다. 그렇기 때문에 각 파일에 대한 정보를 나누어서 따로 저장하도록 하겠습니다. 

In [21]:
import sys, csv, re, glob, os

In [22]:
with open("/scratch/kaldi/egs/INSTRUCTIONAL/exp/test_aligned/final_ali.txt", "r") as fin:
    data = fin.readlines()
    file_name = data[0].strip().split()[0]
    results = []
    for line in data:
        name = file_name
        line = line.strip()
        file_name, utt, start, end, idx, phn = line.split()
        if (file_name != name):
            try:
                with open("exp/test_aligned/align_phones/" + (name)+".txt",'w') as fwrite:
                    writer = csv.writer(fwrite)
                    fwrite.write("\n".join(results))
                    fwrite.close()
                #print name
            except Exception, e:
                print "Failed to write file",e
                sys.exit(2)
            del results[:]
            results.append(line)
        else:
            results.append(line)

try:
    with open("exp/test_aligned/align_phones/" + (name)+".txt",'w') as fwrite:
        writer = csv.writer(fwrite)
        fwrite.write("\n".join(results))
        fwrite.close()
                #print name
except Exception, e:
    print "Failed to write file",e
    sys.exit(2)

위 코드를 실행하시면 `exp/triphones_lda/align_phones` 폴더 아래에 각각의 파일 이름으로 `음소 정보`가 저장되었음을 알 수 있습니다. 

In [23]:
%%bash
ls ${KALDI_INSTRUCTIONAL_PATH}/exp/test_aligned/align_phones | head

1089-134686-0000.txt
1089-134686-0001.txt
1089-134686-0002.txt
1089-134686-0003.txt
1089-134686-0004.txt
1089-134686-0005.txt
1089-134686-0006.txt
1089-134686-0007.txt
1089-134686-0008.txt
1089-134686-0009.txt


파일을 살펴보면 이제는 한 파일에는 하나의 문장에 대한 정보만 들어있는 것을 확인할 수 있습니다. 

In [26]:
%%bash
cat ${KALDI_INSTRUCTIONAL_PATH}/exp/test_aligned/align_phones/1089-134686-0000.txt

1089-134686-0000 1 0.000 0.51 1 SIL
1089-134686-0000 1 0.510 0.57 138 HH_B
1089-134686-0000 1 0.570 0.64 159 IY1_E
1089-134686-0000 1 0.640 0.71 138 HH_B
1089-134686-0000 1 0.710 0.85 196 OW1_I
1089-134686-0000 1 0.850 0.92 216 P_I
1089-134686-0000 1 0.920 1.0 231 T_E
1089-134686-0000 1 1.000 1.05 90 DH_B
1089-134686-0000 1 1.050 1.08 100 EH1_I
1089-134686-0000 1 1.080 1.14 219 R_E
1089-134686-0000 1 1.140 1.21 266 W_B
1089-134686-0000 1 1.210 1.24 244 UH1_I
1089-134686-0000 1 1.240 1.31 87 D_E
1089-134686-0000 1 1.310 1.37 78 B_B
1089-134686-0000 1 1.370 1.42 159 IY1_E
1089-134686-0000 1 1.420 1.55 222 S_B
1089-134686-0000 1 1.550 1.62 232 T_I
1089-134686-0000 1 1.620 1.83 255 UW1_E
1089-134686-0000 1 1.830 1.86 1 SIL
1089-134686-0000 1 1.860 1.89 130 F_B
1089-134686-0000 1 1.890 1.97 107 ER0_E
1089-134686-0000 1 1.970 2.05 86 D_B
1089-134686-0000 1 2.050 2.1 148 IH1_I
1089-134686-0000 1 2.100 2.16 184 N_I
1089-134686-0000 1 2.160 2.36 107 ER0_E
1089-134686-0000 1 2.360 2.58 1 SIL
108

## Align pronunciations

앞에서 생성한 파일을 토대로 우리는 "파일이름/문장 번호/시작 시간/종료 시간/단어의 발음"의 정보를 담은 파일들을 생성할 수 있습니다. 이전 기억을 더듬어 보시면, `kaldi`에서는 음소를 표기할 때 음소의 단어 내 위치까지도 표시하였다는 것을 아실 수 있습니다(**`BIES`**). 우리는 해당 정보를 이용하여서 어떤 음소에서 어떤 음소까지가 하나의 단어에 대한 발음인지를 알 수 있습니다. 아래 셀을 실행하여서 결과를 확인하도록 하겠습니다. 

In [27]:
path = "/scratch/kaldi/egs/INSTRUCTIONAL/exp/test_aligned/align_phones"
pron=[]
files = os.listdir(path)
for fname in files:
    file_name = path + "/" + fname
    f = open(file_name, "r")
    data = f.readlines()
    fout = open("/scratch/kaldi/egs/INSTRUCTIONAL/exp/test_aligned/align_prons/" + fname, "w")
    wstart = 0
    wend = 0
    for line in data:
        line = line.strip()
        name, utt, start, end, idx, phone = line.split()
        if (phone == "SIL"):
            phone = "SIL_S"
        phn, pos = phone.split("_")
        if pos == "B":
            w_start = start
            pron.append(phn)
        if pos == "S":
            w_start=start
            w_end=end
            pron.append(phn)
            fout.write(name + "\t" + " ".join(pron) +"\t"+ str(w_start) + "\t" + str(w_end) + "\n")
            pron=[]
        if pos == "E":
            w_end=end
            pron.append(phn)
            fout.write(name + "\t" + " ".join(pron) +"\t"+ str(w_start) + "\t" + str(w_end) + "\n")
            pron=[]
        if pos == "I":
            pron.append(phn)
    fout.close()
    f.close()

`exp/triphones_lda/align_prons` 디렉토리를 확인하시면 아까와 같은 이름의 파일들이 있는 것을 확인하실 수 있습니다. 

In [29]:
%%bash
ls exp/test_aligned/align_prons/ | head

1089-134686-0000.txt
1089-134686-0001.txt
1089-134686-0002.txt
1089-134686-0003.txt
1089-134686-0004.txt
1089-134686-0005.txt
1089-134686-0006.txt
1089-134686-0007.txt
1089-134686-0008.txt
1089-134686-0009.txt


ls: write error: Broken pipe


파일의 내용은 어떤지 살펴보겠습니다. 

In [30]:
%%bash
head exp/test_aligned/align_prons/1089-134686-0000.txt
tail exp/test_aligned/align_prons/1089-134686-0000.txt

1089-134686-0000	SIL	0.000	0.51
1089-134686-0000	HH IY1	0.510	0.64
1089-134686-0000	HH OW1 P T	0.640	1.0
1089-134686-0000	DH EH1 R	1.000	1.14
1089-134686-0000	W UH1 D	1.140	1.31
1089-134686-0000	B IY1	1.310	1.42
1089-134686-0000	S T UW1	1.420	1.83
1089-134686-0000	SIL	1.830	1.86
1089-134686-0000	F ER0	1.860	1.97
1089-134686-0000	D IH1 N ER0	1.970	2.36
1089-134686-0000	IH0 N	7.560	7.69
1089-134686-0000	SIL	7.690	7.79
1089-134686-0000	TH IH1 K	7.790	8.12
1089-134686-0000	SIL	8.120	8.22
1089-134686-0000	P EH1 P ER0 D	8.220	8.63
1089-134686-0000	SIL	8.630	8.75
1089-134686-0000	F L AW1 ER0	8.750	9.09
1089-134686-0000	F AE1 T AH0 N D	9.090	9.53
1089-134686-0000	S AO1 S	9.530	10.11
1089-134686-0000	SIL	10.110	10.42


이전 파일과 유사하지만, 이제는 음성 파일 어디에서부터 어디까지 어떠한 단어가 발음되었는지의 정보를 담고 있는 것을 볼 수 있습니다. 실제 단어는 어떤 단어일지 아직 모르지만, 그 단어가 **`어떤 발음`**으로 발화되었는지의 정보는 여기에서 확인 가능한 것입니다. 

## Extract real words

이 `notebook`에서 우리는 단어를 예측해야 하는 것이 아니라, 실제 어떠한 단어가 발화되었는지 알고 있습니다. 그렇기 때문에 아래 스크립트를 이용하여서 각각의 파일에서 어떤 단어들이 발화되었는지 순서대로 기록하도록 하겠습니다. 

In [31]:
with open("raw_data/librispeech-transcripts.txt", "r") as f:
    data = f.readlines()
    for line in data:
        line = line.strip()
        info = line.split()
        fname = info[0]
        file_name = "exp/test_aligned/align_transcripts/" + fname + ".txt"
        
        fout = open(file_name, "w")
        for item in info[1:]:
            result = fname + " " + item + "\n"
            fout.write(result)
        fout.close()
    f.close()

In [32]:
%%bash 
ls exp/test_aligned/align_transcripts | grep "1089-134686-0000"

1089-134686-0000.txt


생성된 파일을 확인해보면, `SIL`을 제외하고는 실제 문장에서 어떤 단어들이 발화되었는지 순서대로 나타나 있는 것을 확인할 수 있습니다. 

In [33]:
%%bash
head exp/test_aligned/align_transcripts/1089-134686-0000.txt

1089-134686-0000 HE
1089-134686-0000 HOPED
1089-134686-0000 THERE
1089-134686-0000 WOULD
1089-134686-0000 BE
1089-134686-0000 STEW
1089-134686-0000 FOR
1089-134686-0000 DINNER
1089-134686-0000 TURNIPS
1089-134686-0000 AND


## pron2word

`kaldi`를 시작할 때, 우리는 발음 사전을 이용했었습니다. 같은 파일을 이용해서 이번에는 `발음-단어`의 관계를 정의하는 `<dict>`를 만들어 보도록 하겠습니다. 이 과정을 수행하는 목적은 어떠한 단어가 발음되었을 때, 그 발음을 가진 단어들을 **후보군**으로 지정하고 나중에 실제 문장에서 발화된 단어와 일치하는지 확인하기 위해서입니다. 

In [34]:
pron2word = {}
with open("/scratch/kaldi/egs/INSTRUCTIONAL/data/local/dict/lexicon.txt", "r") as f:
    data = f.readlines()
    for line in data:
        line = line.strip()
        info = line.split()
        word = info[0]
        pron = " ".join(info[1:])
        try:
            pron2word[pron].append(word)
        except:
            pron2word[pron] = []
            pron2word[pron].append(word)
    f.close
print(pron2word["EY1 Z"])

["A''S", "A'S", 'AISE', 'AYS', 'EYS']


`pron2word`와 앞서 생성된 `align_prons`, `align_transcripts`를 이용하여서 "파일 이름/단어/시작 시간/종료 시간"의 정보를 담고 있는 파일들을 생성해보도록 하겠습니다. 먼저 `align_prons` 파일을 읽어온 이후, 같은 이름을 가진 `align_transcripts`를 읽어옵니다. `align_prons`의 파일들은 `SIL`을 제외하고는 `align_transcript` 폴더에 같은 이름을 가진 파일의 단어들이 순서대로 발화되어야 하므로, `align_prons`에 있는 발음들의 **후보군** 단어들을 불러오고, `align_transcript`에 같은 순서의 단어가 후보군에 있는지 확인하는 과정을 거쳤습니다. 후보군에 해당 단어가 있는 경우 실제 단어를 출력하고, 잘못 발화되었을 경우 `<UNK>`를 출력하는 형식입니다. 

In [35]:
pron_path = "/scratch/kaldi/egs/INSTRUCTIONAL/exp/test_aligned/align_prons"
tran_path = "/scratch/kaldi/egs/INSTRUCTIONAL/exp/test_aligned/align_transcripts"

pron_files = os.listdir(pron_path)
tran_files = os.listdir(tran_path)
for fname in pron_files:
    pron_name = pron_path + "/" + fname
    tran_name = tran_path + "/" + fname
    word_name = "/scratch/kaldi/egs/INSTRUCTIONAL/exp/test_aligned/align_words/" + fname
    f_pron = open(pron_name, "r")
    f_tran = open(tran_name, "r")
    pron_data = f_pron.readlines()
    tran_data = f_tran.readlines()
    
    fout = open(word_name, "w")
    
    idx = 0
    for line in pron_data:
        line = line.strip()
        name, pron, start, end = line.split("\t")
        if (pron == "SIL"):
            result = line + "\n"
            fout.write(result)
        else:
            candidates = pron2word[pron]
            info = tran_data[idx]
            t_name, t_word = info.split()
            if (t_word in candidates):
                result = name + "\t" + t_word + "\t" + start + "\t" + end + "\n"
                fout.write(result)
                idx += 1
            else:
                result = name + "\t<UNK>\t" + start + "\t" + end + "\n"
                fout.write(result)
                idx += 1
    f_pron.close()
    f_tran.close()
    fout.close()                

In [36]:
%%bash 
ls exp/test_aligned/align_words | head

1089-134686-0000.txt
1089-134686-0001.txt
1089-134686-0002.txt
1089-134686-0003.txt
1089-134686-0004.txt
1089-134686-0005.txt
1089-134686-0006.txt
1089-134686-0007.txt
1089-134686-0008.txt
1089-134686-0009.txt


실행 결과를 살펴보면 단어 정보가 제대로 옮겨졌다는 것을 볼 수 있습니다. 

In [37]:
%%bash 
head exp/test_aligned/align_words/1089-134686-0000.txt
tail exp/test_aligned/align_words/1089-134686-0000.txt

1089-134686-0000	SIL	0.000	0.51
1089-134686-0000	HE	0.510	0.64
1089-134686-0000	HOPED	0.640	1.0
1089-134686-0000	THERE	1.000	1.14
1089-134686-0000	WOULD	1.140	1.31
1089-134686-0000	BE	1.310	1.42
1089-134686-0000	STEW	1.420	1.83
1089-134686-0000	SIL	1.830	1.86
1089-134686-0000	FOR	1.860	1.97
1089-134686-0000	DINNER	1.970	2.36
1089-134686-0000	IN	7.560	7.69
1089-134686-0000	SIL	7.690	7.79
1089-134686-0000	THICK	7.790	8.12
1089-134686-0000	SIL	8.120	8.22
1089-134686-0000	PEPPERED	8.220	8.63
1089-134686-0000	SIL	8.630	8.75
1089-134686-0000	FLOUR	8.750	9.09
1089-134686-0000	FATTENED	9.090	9.53
1089-134686-0000	SAUCE	9.530	10.11
1089-134686-0000	SIL	10.110	10.42


## Create TextGrid files

이제는 생성한 모든 파일들을 토대로 `TextGrid` 파일을 생성해보도록 하겠습니다. 

과정이 매우 복잡해 보일 수 있지만, 대부분의 과정은 `TextGrid` 파일을 `Praat`에서 제대로 읽어오기 위한 정보를 담고 있는 부분들이 많습니다. 

내부에서 phone, word, 그리고 sentence를 각각의 티어에 담기 위해 진행되는 과정은 단순합니다. 

 1. align_words 파일을 이용하여 전체 음성 파일의 시작/종료 시간 구하기
 2. align_phones 파일의 각 줄을 읽어오면서 시작 시간, 종료 시간, 음소를 출력
 3. align_words 파일의 각 줄을 읽어오면서 시작 시간, 종료 시간, 음소를 출력
 4. align_words 파일에서 "SIL"을 제외한 단어들을 이용하여 문장을 구성해고, 해당 문장을 마지막 티어로 추가. 

In [38]:
pron_path = "/scratch/kaldi/egs/INSTRUCTIONAL/exp/test_aligned/align_phones"
word_path = "/scratch/kaldi/egs/INSTRUCTIONAL/exp/test_aligned/align_words"

pron_files = os.listdir(pron_path)
word_files = os.listdir(word_path)
for fname in pron_files:
    name, ext = fname.split(".")
    pron_name = pron_path + "/" + fname
    word_name = word_path + "/" + fname
    tg_name = "/scratch/kaldi/egs/INSTRUCTIONAL/exp/test_aligned/textgrid/" + name + ".TextGrid"
    f_pron = open(pron_name, "r")
    f_word = open(word_name, "r")
    pron_data = f_pron.readlines()
    word_data = f_word.readlines()
    
    f_start = word_data[0].strip().split("\t")[2]
    f_end = word_data[-1].strip().split("\t")[3]
    
    fout = open(tg_name, "w")
    
    fout.write('File type = "ooTextFile"\n')
    fout.write('Object class = "TextGrid"\n\n')
    fout.write("xmin = " + str(f_start) + "\n")
    fout.write("xmax = " + str(f_end) + "\n")
    fout.write("tiers? <exists>\n")
    fout.write("size = 3\n")
    fout.write("item []:\n")
    
    fout.write("\titem [1]:\n")
    fout.write('\t\tclass = "IntervalTier"\n')
    fout.write('\t\tname = "phone"\n')
    fout.write("\t\txmin = " + str(f_start) + "\n")
    fout.write("\t\txmax = " + str(f_end) + "\n")
    fout.write("\t\tintervals: size = " + str(len(pron_data)) + "\n")
    for i in range(0, len(pron_data)):
        fout.write("\t\tintervals [" + str(i + 1) + "]:\n")
        info = pron_data[i].strip().split()
        start = info[2]
        end = info[3]
        pron = info[5]
        if (pron != "SIL"):
            phn, pos = pron.split("_")
            pron = phn
        fout.write("\t\t\txmin = " + start + "\n")
        fout.write("\t\t\txmax = " + end + "\n")
        fout.write('\t\t\ttext = "' + pron + '"\n')
    fout.write("\titem [2]:\n")
    fout.write('\t\tclass = "IntervalTier"\n')
    fout.write('\t\tname = "word"\n')
    fout.write("\t\txmin = " + str(f_start) + "\n")
    fout.write("\t\txmax = " + str(f_end) + "\n")
    fout.write("\t\tintervals: size = " + str(len(word_data)) + "\n")
    words = []
    for i in range(0, len(word_data)):
        fout.write("\t\tintervals [" + str(i + 1) + "]:\n")
        info = word_data[i].strip().split()
        start = info[2]
        end = info[3]
        word = info[1]
        if word != "SIL":
            words.append(word) 
        fout.write("\t\t\txmin = " + start + "\n")
        fout.write("\t\t\txmax = " + end + "\n")
        fout.write('\t\t\ttext = "' + word + '"\n')
    fout.write("\titem [3]:\n")
    fout.write('\t\tclass = "IntervalTier"\n')
    fout.write('\t\tname = "sent"\n')
    fout.write("\t\txmin = " + str(f_start) + "\n")
    fout.write("\t\txmax = " + str(f_end) + "\n")
    fout.write("\t\tintervals: size = 1\n")
    fout.write("\t\tintervals [1]:\n")
    info = word_data[i].strip().split()
    start = info[2]
    end = info[3]
    sent = " ".join(words)
    fout.write("\t\t\txmin = " + str(f_start) + "\n")
    fout.write("\t\t\txmax = " + str(f_end) + "\n")
    fout.write('\t\t\ttext = "' + sent + '"\n')
    f_pron.close()
    f_word.close()
    fout.close()                

결과로 다음과 같은 `TextGrid` 파일들이 생성됩니다. 

In [41]:
%%bash
ls exp/test_aligned/textgrid | head

1089-134686-0000.TextGrid
1089-134686-0001.TextGrid
1089-134686-0002.TextGrid
1089-134686-0003.TextGrid
1089-134686-0004.TextGrid
1089-134686-0005.TextGrid
1089-134686-0006.TextGrid
1089-134686-0007.TextGrid
1089-134686-0008.TextGrid
1089-134686-0009.TextGrid


파일을 훑어보면 실제 TextGrid 파일과 동일한 구조로 되어있음을 알 수 있습니다. 

In [42]:
%%bash
head -n 30 exp/test_aligned/textgrid/1089-134686-0000.TextGrid

File type = "ooTextFile"
Object class = "TextGrid"

xmin = 0.000
xmax = 10.42
tiers? <exists>
size = 3
item []:
	item [1]:
		class = "IntervalTier"
		name = "phone"
		xmin = 0.000
		xmax = 10.42
		intervals: size = 113
		intervals [1]:
			xmin = 0.000
			xmax = 0.51
			text = "SIL"
		intervals [2]:
			xmin = 0.510
			xmax = 0.57
			text = "HH"
		intervals [3]:
			xmin = 0.570
			xmax = 0.64
			text = "IY1"
		intervals [4]:
			xmin = 0.640
			xmax = 0.71
			text = "HH"


sample 파일을 실제 `Praat`에서 열어보도록 하겠습니다. 